# Ingest results from bioinformatics run

`testuser1` performed perturb-seq and initiated a bioinformatics workflow run:

In [ ]:
!lndb login testuser1

In [ ]:
import lamindb as ln

## Upload raw fastq files

In [ ]:
bfx_run_output = ln.dev.datasets.dir_scrnaseq_cellranger(
    "perturbseq", basedir=ln.settings.storage, output_only=False
)
ln.track(ln.Transform(name="Chromium 10x upload", type="pipeline"))
ln.File(bfx_run_output.parent / "fastq/perturbseq_R1_001.fastq.gz").save()
ln.File(bfx_run_output.parent / "fastq/perturbseq_R2_001.fastq.gz").save()
ln.setup.login("testuser2")

## Bioinformatics run

In [ ]:
transform = ln.Transform(name="Cell Ranger", version="7.2.0", type="pipeline")
ln.track(transform)

In [ ]:
files = ln.File.filter(key__startswith="fastq/perturbseq").all()
filepaths = [file.stage() for file in files]

In [ ]:
output_files = ln.File.from_dir("./mycrispr/perturbseq/filtered_feature_bc_matrix/")
ln.save(output_files)

In [ ]:
output_files[0].view_lineage()